# Theory - Wall Heat Balance

In order to accurately model the heat flux between fluid and walls, we resolve the local temperature profile $T(z)$ at each cell $(x, y)$ in the domain.

The following sections describe how this temperature profile can be computed from the conserved, height-averaged quantities in the macro solution.

## Derivation from Microproblem

The full state vector and flux matrix are
$$
\vec{q} =
\begin{pmatrix}
\rho \\ \vec{j} \\ E
\end{pmatrix},\qquad
\underline{F} =
\begin{pmatrix}
\vec{j} \\
\frac{1}{\rho}\vec{j}\otimes\vec{j} + p\underline{1} - \underline{\tau} \\
E\frac{\vec{j}}{\rho}
- \underline{\sigma}\frac{\vec{j}}{\rho}
- k\nabla T
\end{pmatrix}.
$$

We are interested in the $z$-component of the energy flux:
$$
f_{z,E} =
E\frac{j_z}{\rho}
- (\tau_{xz}u + \tau_{yz}v + \tau_{zz}w)
- k\frac{\partial T}{\partial z}.
$$

For the micro-problem, we demand (see Microscale Problem in [Governing Equations](./01_macro_equations.ipynb)):
$$
\frac{\partial}{\partial z} f_{z,E} = \text{const}.
$$

With $j_z=0$:
$$
\frac{\partial}{\partial z}\Big(
\tau_{xz}u + \tau_{yz}v + k\frac{\partial T}{\partial z}
\Big)
= \text{const}.
$$

Using $\tau_{xz}=\mu\,\partial_z u$, $\tau_{yz}=\mu\,\partial_z v$, the microproblem reduces to
$$
\mu\!\left[(\partial_z u)^2 + (\partial_z v)^2\right] + k\,\partial_{zz}T = \text{const}
$$

This leaves us with three degrees of freedom for $T(z)$. In the following we denote the `const` value as $\overline{s}$.

---

## Analytic Solution

The analytic solution is illustrated for the 1D case (neglecting $v(z)$).

Using the quadratic velocity profile from the flow micro problem:
$$
u(z)=\alpha z^2+\beta z+\gamma,\qquad u'(z)=2\alpha z+\beta,
$$
so
$$
u'(z)^2 = 4\alpha^2 z^2 + 4\alpha\beta z + \beta^2.
$$

Inserting this in the derived ODE from above:
$$
0 = T''(z) + \frac{\mu}{k}\big(4\alpha^2 z^2 + 4\alpha\beta z + \beta^2\big) + \overline{s}.
$$

Integrating twice with integration constants $C_1, C_2$ gives
$$
T(z) = \frac{\overline{s}}{2}z^2 + C_1 z + C_2
- \frac{\mu}{k}\!\left(\frac{4\alpha^2}{12}z^4 + \frac{4\alpha\beta}{6}z^3 + \frac{\beta^2}{2}z^2\right),
$$
unknowns: $C_1,\;C_2,\;\overline{s}$, therefore three equations are required to full definition.

---

## Closure Conditions

Determine $(C_1,C_2,\overline{s})$ from:

1. Robin BC at upper wall ($z=h_2$):
$$
- k\,T'(h_2) \;=\; h_{\mathrm{eff}}\big(T(h_2)-T_{\text{bulk,top}}\big).
$$

2. Robin BC at lower wall ($z=h_1$):
$$
- k\,T'(h_1) \;=\; h_{\mathrm{eff}}\big(T(h_1)-T_{\text{bulk,bot}}\big).
$$

3. Known height-averaged temperature:
$$
\overline{T} \;=\; \frac{1}{h}\int_{h_1}^{h_2} T(z)\,dz.
$$

These three equations fully define $T(z)$.

## Meaning of the Source Term

Writing down the "full" inner energy equation assuming a 2D problem (flow in $x$-direction, film thickness across $z$, no flow in $z$-direction)

$$
\frac{\partial E_{\mathrm{inner}}}{\partial t} =
k\,T_{zz}
+ k\,T_{xx}
+ \mu\left(\frac{\partial u}{\partial z}\right)^2
+ \mu\left(\frac{\partial u}{\partial x}\right)^2
- u\,c_v\,T_x ,
$$

allows the interpretation that $\overline{s}$ "covers" energy advection, temperature diffusion, and shear heating in $x$-direction:

$$
\overline{s} \approx \frac{1}{h} \int_{h_1}^{h_2} \left[ k\,T_{xx} - u\,c_v\,T_x + \mu\left(\frac{\partial u}{\partial x}\right)^2 \right] dz
$$

But note that the source additionally covers time-dependency. Here are two use cases for further illustration of what the source can depict:

---

### Case A — Steady Poiseuille Flow

If the flow is significant and the temperature varies along the flow direction,

$$s \approx -u\,c_v\,T_x.$$

$\overline{s}$ represents **advective energy transport** entering or leaving the local film slice.

---

### Case B — Stored Energy in Transient Simulation

Imagine there is no flow and initial temperature of the fluid differs from the wall temperature.

In this case, the source represents **stored internal energy** that is being released or absorbed as the macro field relaxes.  
As the macro-system approaches steady state, temporal storage vanishes and $\overline{s} \rightarrow 0$.
